## 初步验证

### 1. CoT-任务拆解能力

In [ ]:
prompt = '''
你现在是一个金融领域专家，我们有一系列原子化api，负责在收到用户的query后你需要通过编排api以供我们后续调用获得结果（你不负责调用，只需要生成调用序列即可），输出json格式。
query是：我打算用100万元买三羊马的股票，如果按照三羊马的最高价来计算，我能买多少股呢？
query中提到的产品标准名可能是：三羊马
所有可用api:
    基金查询类：(查询某基金的以下指标)
        代码（输入名称查股票）,类型,规模,晨星评级,蚂蚁评级,申购费率,成立年限,开放类型,申购状态,蚂蚁金选标识,风险等级,基金经理类型,基金经理,基金公司,基金经理从业年限,基金经理管理规模,基金经理年化回报率,持股估值,持股集中度,持股热门度,持股换手率,机构占比,单一持有人占比,重仓股票,重仓行业,持仓风格,基金公司是否自购,是否可投沪港深,近一年买入持有6个月历史盈利概率,近3年买入持有1年历史盈利概率,近5年买入持有1年历史盈利概率,近期收益率,近期收益率同类排名,近期年化收益率,近期年化收益率同类排名,近期超大盘收益率,近期创新高次数,近期最大回撤,近期最大回撤同类排名,近期波动率同类排名,近期最长解套天数,近期跟踪误差,近期夏普比率同类排名,近期卡玛比率同类排名,近期信息比率同类排名,年度收益率,近期指数增强,年度最大回撤,基金事后分类,基金资产打分,基金募集类型,投组区域,基金份额类型,申购最小起购金额,分红方式,赎回状态,是否支持定投,单位净值同步日期,单位净值,基金经理任职期限,近期夏普率,投资市场,板块,行业,聚类行业,管理费率,销售服务费率,托管费率,认购费率,赎回费率,分红年度,权益登记日,除息日,派息日,红利再投日,每十份收益单位派息（元）
    条件选基类：(根据以下指标选择基金)
        除第一个外（本类第一个为输入基金代码，查询详细信息），其余与基金查询类api名称相同（一一对应），区别是基金查询类api是输入基金代码，查询相应的指标值；而条件选基api是输入指标（如“规模”）和运算符（如“>” “<” “=” "包括"等），输出符合该指标对应条件的基金列表。
    股票查询类：
        代码,开盘价,最高价,最低价,当前价,收盘价,成交量,成交额,涨停价,跌停价,涨跌额,涨跌幅,换手率,每股收益,静态市盈率,总市值,振幅,流通市值,每股收益ttm,市盈率ttm,净资产收益率,每股净资产,每股经营性现金流,毛利率,净利率,净利润,净利润同比增长,营业收入,营收同比增长,投资收入,营业利润,营业利润同比增长,扣非净利润,流动资产,总资产,短期负债,总负债,股东权益,净经营性现金流,净投资性现金流,净融资性现金流,银行资本充足率,流通股本,总股本,主力资金流入,主力资金流出,主力资金净流入,总流入,总流出,高管名称
    条件选股类：
        与条件选基一样。
    通用运算符：
        加法计算,减法计算,乘法计算,除法计算,与计算,或计算

示例：
    query是：我想了解汇丰晋信2016C的基金经理是谁，目前的基金规模以及申购费率是多少?
    query中提到的产品标准名可能是：汇丰晋信2016生命周期开放式证券投资基金C类
    回答(你的输出示例，下文中缩进的内容)：
        思考过程:
            首先我们需要查找 汇丰晋信2016生命周期开放式证券投资基金C类 的基金代码
                调用 api_0: (基金查询类-代码) 获取基金代码
            然后根据 代码 查询 对应的基金经理
                调用 api_1 (基金查询类-查询基金经理) 获取基金经理
            同样，根据 代码 查询对应的基金规模：
                调用 api_2 (基金查询类-查询基金规模) 获取基金规模
            最后，还要查找该基金的申购费率
                调用 api_3 (基金查询类-查询申购费率) 获取基金申赎费率
            根据问题，最终需要输出（api_1,api_2,api_3）的调用结果
        结果
            api_list:[基金查询类-代码,基金查询类-查询基金经理,基金查询类-查询基金规模,基金查询类-查询申购费率]
    
    让我们一步一步来，写出思考过程，并在最后汇总成格式化的答案
    注意：1. 严格按照上面的示例格式回答；2. 只能在上文中”所有可用的api“中选api，不能捏造
'''

# 优化：先通过

In [ ]:
以下是GLM-4的输出结果

```
思考过程:
    首先我们需要查找 三羊马 的股票代码
        调用 api_0: (股票查询类-代码) 获取股票代码
    然后根据 股票代码 查询 对应的最高价
        调用 api_1 (股票查询类-最高价) 获取最高价
    根据用户的投资金额和股票的最高价，我们可以计算出用户可以购买的股票数量：
        股票数量 = 投资金额 / 最高价
    由于我们需要进行除法计算，我们将使用通用运算符中的除法计算API来得到最终的股票数量
        调用 api_2: (通用运算符-除法计算) 进行计算
    根据问题，最终需要输出股票数量

结果
    api_list:[股票查询类-代码,股票查询类-最高价,通用运算符-除法计算]
```

### 2. 纠错能力

In [ ]:
错误类型

# 规则验证？ 
    # 行业知识引入
        如：股票、基金的买卖，都涉及哪些费用
    # 基本
        价格查询：
    

In [ ]:
prompt = '''
    领域知识：
        。。

'''